ESPN 迴歸分析

In [3]:
import sys
sys.path.append('G:\我的雲端硬碟\python')
import numpy as np
from polib.CsvEngn import *
from MLB_functions import *
from tqdm import tqdm

import statsmodels.api as sm #OLS
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html

from linearmodels.panel import PanelOLS # FE
# https://bashtage.github.io/linearmodels/doc/panel/models.html

僅挑選 have_team資料

In [22]:
# 0530

#投手
df_pitch = pd.read_excel(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\3.變數\excel_data\ESPN_haveTeam_pitch_20210530_13.27.48.xlsx",
                        engine='openpyxl')
# 野手
df_bat = pd.read_excel(r"G:\我的雲端硬碟\learning\論文今年會寫完\0.資源包\3.變數\excel_data\ESPN_haveTeam_bat_20210530_13.27.38.xlsx",
                      engine='openpyxl')

In [9]:
#T,F transform
for df in [df_pitch, df_bat]:
    for col in ["is_FA_contract_year", "FIRST"]:
        df[col] = df[col].apply(lambda v: 1 if v else 0)
        
# rename columns        
df_pitch.rename(columns={"POS_c":"POS",
                         "season":"season_org",
                         "season_num":"season",
                        "playerID":"playerID_org",
                         "playerID_num":"playerID"
                        }, inplace=True)

df_bat.rename(columns={"POS_c":"POS",
                         "season":"season_org",
                         "season_num":"season",
                        "playerID":"playerID_org",
                         "playerID_num":"playerID"
                        }, inplace=True)

In [10]:
df_pitch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2322 entries, 0 to 2321
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   _idx                 2322 non-null   int64  
 1   season_org           2322 non-null   int64  
 2   season               2322 non-null   int64  
 3   Team                 2322 non-null   object 
 4   GP                   2322 non-null   int64  
 5   GS                   2322 non-null   int64  
 6   WAR                  2322 non-null   float64
 7   ERA                  2322 non-null   float64
 8   IP                   2322 non-null   float64
 9   K/BB                 2322 non-null   float64
 10  playerID_org         2322 non-null   int64  
 11  playerID             2322 non-null   int64  
 12  Yrs                  2322 non-null   int64  
 13  TYPE                 2322 non-null   object 
 14  POS                  2322 non-null   object 
 15  Age                  2322 non-null   i

In [11]:
df_bat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3007 entries, 0 to 3006
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   _idx                 3007 non-null   int64  
 1   season_org           3007 non-null   int64  
 2   season               3007 non-null   int64  
 3   Team                 3007 non-null   object 
 4   GP                   3007 non-null   int64  
 5   WAR                  3007 non-null   float64
 6   playerID_org         3007 non-null   int64  
 7   playerID             3007 non-null   int64  
 8   Yrs                  3007 non-null   int64  
 9   AB                   3007 non-null   int64  
 10  OPS                  3007 non-null   float64
 11  TYPE                 3007 non-null   object 
 12  POS                  3007 non-null   object 
 13  Age                  3007 non-null   int64  
 14  Age_sq               3007 non-null   int64  
 15  YEAR                 1066 non-null   f

## X Split

In [12]:
X_pitch_cols = ["playerID", "season", "is_FA_contract_year", "FIRST" ,"END", "GP",  "Age",  "Age_sq", "Team", "POS"]
df_pitch_X = df_pitch[X_pitch_cols].copy(deep=True)

X_bat_cols = ["playerID", "season","is_FA_contract_year", "FIRST" ,"END", "GP",  "Age",  "Age_sq", "Team", "POS"]
df_bat_X = df_bat[X_bat_cols].copy(deep=True)

## Get dummy: Team、POS

In [13]:
def set_dummy(df, dummy_lst):
    for col in dummy_lst:
        dummy_cols = pd.get_dummies(df[col], prefix=col)
        dummy_cols.drop(dummy_cols.columns[-1], axis = 1, inplace=True) # 刪除最後一欄 避免變數共線性問題
        df = pd.concat([df, dummy_cols], axis=1)
    return df
    
dummy_lst = ["Team","POS"]
df_pitch_X = set_dummy(df_pitch_X, dummy_lst)
df_pitch_X.drop(dummy_lst, axis = 1, inplace=True)


df_bat_X = set_dummy(df_bat_X, dummy_lst)
df_bat_X.drop(dummy_lst, axis = 1, inplace=True)

ID_cols = ["playerID", "season"]

## OLS、FE(+season_num)

In [14]:
# OLS
y = df_pitch.WAR.copy(deep=True)
X = df_pitch_X.copy(deep=True)
X.drop(ID_cols, axis = 1, inplace=True) # drop ID, season
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.193
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     15.16
Date:                Sun, 30 May 2021   Prob (F-statistic):           1.30e-81
Time:                        14:09:47   Log-Likelihood:                -4186.3
No. Observations:                2322   AIC:                             8447.
Df Residuals:                    2285   BIC:                             8659.
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   7.3731      1.965      3.751      0.000       3.519      11.227
is_FA_contract_year    -0.4089      0.069     -5.907      0.000      -0.545      -0.273
FIRST                  -0.1435      0.112     -1.287      0.198      -0.362       0.075
END                    -0.7610      0.106     -7.158      0.000      -0.969      -0.552
GP                      0.0253      0.002     12.111      0.000       0.021       0.029
Age                    -0.3132      0.113     -2.766      0.006      -0.535      -0.091
Age_sq                  0.0042      0.002      2.566      0.010       0.001       0.007
Team_ARI               -0.5785      0.236     -2.452      0.014      -1.041      -0.116
Team_ATL               -0.2415      0.246     -0.982      0.326      -0.724       0.241
Team_BAL               -0.9115      0.238     -3.834      0.000      -1.378      -0.445
Team_BOS               -0.5980      0.218     -2.741      0.006      -1.026      -0.170
Team_CHC               -0.3943      0.217     -1.814      0.070      -0.821       0.032
Team_CHW               -0.2411      0.234     -1.030      0.303      -0.700       0.218
Team_CIN               -0.7022      0.240     -2.928      0.003      -1.172      -0.232
Team_CLE               -0.7364      0.251     -2.938      0.003      -1.228      -0.245
Team_COL               -0.5881      0.226     -2.599      0.009      -1.032      -0.144
Team_DET               -0.6959      0.244     -2.846      0.004      -1.175      -0.216
Team_HOU               -0.4161      0.231     -1.799      0.072      -0.870       0.037
Team_KC                -0.6239      0.230     -2.709      0.007      -1.075      -0.172
Team_LAA               -0.6409      0.228     -2.805      0.005      -1.089      -0.193
Team_LAD               -0.4362      0.227     -1.925      0.054      -0.881       0.008
Team_MIA               -0.8146      0.257     -3.166      0.002      -1.319      -0.310
Team_MIL               -0.9055      0.227     -3.985      0.000      -1.351      -0.460
Team_MIN               -0.4123      0.228     -1.809      0.071      -0.859       0.035
Team_NYM               -0.6916      0.228     -3.037      0.002      -1.138      -0.245
Team_NYY               -0.0900      0.221     -0.408      0.683      -0.522       0.342
Team_OAK               -0.3299      0.250     -1.317      0.188      -0.821       0.161
Team_PHI               -0.3435      0.220     -1.562      0.118      -0.775       0.088
Team_PIT               -0.7722      0.246     -3.144      0.002      -1.254      -0.291
Team_SD                -0.9796      0.233     -4.213      0.000      -1.436      -0.524
Team_SEA               -0.7492      0.245     -3.058      0.002      -1.230      -0.269
Team_SF                -0.7201      0.216     -3.326      0.001      -1.145      -0.296
Tea

In [15]:
# OLS
y = df_bat.WAR.copy(deep=True)
X = df_bat_X.copy(deep=True)
X.drop(ID_cols, axis = 1, inplace=True) # drop ID, season
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    WAR   R-squared:                       0.356
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     40.04
Date:                Sun, 30 May 2021   Prob (F-statistic):          5.30e-249
Time:                        14:09:47   Log-Likelihood:                -5346.1
No. Observations:                3007   AIC:                         1.078e+04
Df Residuals:                    2965   BIC:                         1.103e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                   3.0268      2.102      1.440      0.150      -1.095       7.149
is_FA_contract_year    -0.2974      0.059     -5.083      0.000      -0.412      -0.183
FIRST                  -0.4974      0.101     -4.902      0.000      -0.696      -0.298
END                    -0.5181      0.087     -5.980      0.000      -0.688      -0.348
GP                      0.0213      0.001     30.012      0.000       0.020       0.023
Age                    -0.1990      0.123     -1.618      0.106      -0.440       0.042
Age_sq                  0.0021      0.002      1.146      0.252      -0.001       0.006
Team_ARI                0.0506      0.202      0.250      0.802      -0.346       0.447
Team_ATL                0.2191      0.193      1.135      0.256      -0.159       0.597
Team_BAL                0.2146      0.189      1.136      0.256      -0.156       0.585
Team_BOS                0.6552      0.191      3.432      0.001       0.281       1.030
Team_CHC                0.0794      0.200      0.398      0.691      -0.312       0.471
Team_CHW               -0.1789      0.197     -0.910      0.363      -0.565       0.207
Team_CIN               -0.0942      0.208     -0.454      0.650      -0.501       0.313
Team_CLE                0.1890      0.198      0.956      0.339      -0.199       0.577
Team_COL                0.0370      0.209      0.177      0.859      -0.373       0.447
Team_DET                0.2608      0.193      1.353      0.176      -0.117       0.639
Team_HOU                0.0804      0.209      0.384      0.701      -0.330       0.491
Team_KC                -0.0227      0.197     -0.115      0.908      -0.410       0.364
Team_LAA                0.4415      0.195      2.259      0.024       0.058       0.825
Team_LAD                0.5866      0.185      3.165      0.002       0.223       0.950
Team_MIA               -0.4223      0.218     -1.940      0.052      -0.849       0.005
Team_MIL                0.3521      0.204      1.726      0.084      -0.048       0.752
Team_MIN                0.3596      0.201      1.791      0.073      -0.034       0.753
Team_NYM                0.3937      0.190      2.077      0.038       0.022       0.766
Team_NYY                0.9353      0.181      5.164      0.000       0.580       1.290
Team_OAK                0.1542      0.196      0.785      0.432      -0.231       0.539
Team_PHI                0.3057      0.192      1.593      0.111      -0.071       0.682
Team_PIT                0.1981      0.208      0.955      0.340      -0.209       0.605
Team_SD                 0.2902      0.211      1.377      0.169      -0.123       0.703
Team_SEA                0.6311      0.196      3.223      0.001       0.247       1.015
Team_SF                 0.1313      0.195      0.673      0.501      -0.252       0.514
Tea

In [16]:
# FE
X_fe = df_pitch_X.copy(deep=True)
X_fe = X_fe.set_index(ID_cols)
X_fe = sm.add_constant(X_fe)

y_fe = df_pitch[ID_cols+["WAR"]].copy(deep=True)
y_fe = y_fe.set_index(ID_cols)

mod = PanelOLS(y_fe, X_fe, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

C:\Users\Daguan\miniconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

POS_RP

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.1581
Estimator:                   PanelOLS   R-squared (Between):             -0.4410
No. Observations:                2322   R-squared (Within):               0.1581
Date:                Sun, May 30 2021   R-squared (Overall):             -0.0798
Time:                        14:09:48   Log-likelihood                   -3608.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9.3435
Entities:                         546   P-value                           0.0000
Avg Obs:                       4.2527   Distribution:                 F(35,1741)
Min Obs:                       1.0000                                           
Max Obs:                       15.000   F-statistic (robust):             7.5669
                            

In [17]:
# FE
X_fe = df_bat_X.copy(deep=True)
X_fe = X_fe.set_index(ID_cols)
X_fe = sm.add_constant(X_fe)

y_fe = df_bat[ID_cols+["WAR"]].copy(deep=True)
y_fe = y_fe.set_index(ID_cols)

mod = PanelOLS(y_fe, X_fe, 
               entity_effects=True, 
#                time_effects=True,
               drop_absorbed=True,
               check_rank =False
              )
res = mod.fit(cov_type='clustered', cluster_entity=True,)
print(res)

C:\Users\Daguan\miniconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

POS_1B, POS_2B, POS_3B, POS_C, POS_DH, POS_OF

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3719
Estimator:                   PanelOLS   R-squared (Between):             -0.2604
No. Observations:                3007   R-squared (Within):               0.3719
Date:                Sun, May 30 2021   R-squared (Overall):              0.1793
Time:                        14:09:49   Log-likelihood                   -4556.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      40.262
Entities:                         592   P-value                           0.0000
Avg Obs:                       5.0794   Distribution:                 F(35,2380)
Min Obs:                       1.0000                                           
Max Obs:                       14.000   F-statistic (robust):             30.940
                            

## short cut

WAR sum 分組

In [23]:
player_WAR_agg = df_bat.groupby("playerID", as_index=False)["WAR"].sum()
player_WAR_agg.rename(columns={"WAR":"WAR_sum"}, inplace=True)
df_bat = df_bat.merge(player_WAR_agg, how='left',on='playerID')

In [24]:
df_bat["WAR_class"] = pd.qcut(df_bat.WAR_sum, 5, labels=False)
df_bat["WAR_class"].value_counts()

2    610
0    608
1    602
4    598
3    589
Name: WAR_class, dtype: int64

挑選條件

In [29]:
df_pic_org = df_pitch.copy(deep=True)
df_pic_org = df_bat[df_bat["WAR_class"]<5].copy(deep=True)
is_FA_contract_col = "is_FA_contract_year"
#  is_FA_contract_year_before (不顯著) is_FA_contract_year_after(FE正向顯著)

主程式

In [30]:
#T,F transform
for col in [is_FA_contract_col, "FIRST"]:
    df_pic_org[col] = df_pic_org[col].apply(lambda v: 1 if v else 0)
        
# rename columns        
df_pic = df_pic_org.rename(columns={"POS_c":"POS",
                         "season":"season_org",
                         "season_num":"season",
                        "playerID":"playerID_org",
                         "playerID_num":"playerID"
                        })

X_cols = ["playerID", "season", is_FA_contract_col, "FIRST" ,"END", "GP",  "Age",  "Age_sq", "Team", "POS"]
df_pic_X = df_pic[X_cols].copy(deep=True)

def set_dummy(df, dummy_lst):
    for col in dummy_lst:
        dummy_cols = pd.get_dummies(df[col], prefix=col)
        dummy_cols.drop(dummy_cols.columns[-1], axis = 1, inplace=True) # 刪除最後一欄 避免變數共線性問題
        df = pd.concat([df, dummy_cols], axis=1)
    return df
    
dummy_lst = ["Team","POS"]
df_pic_X = set_dummy(df_pic_X, dummy_lst)
df_pic_X.drop(dummy_lst, axis = 1, inplace=True)

ID_cols = ["playerID", "season"]

# OLS
y = df_pic.WAR.copy(deep=True)
X = df_pic_X.copy(deep=True)
X.drop(ID_cols, axis = 1, inplace=True) # drop ID, season
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
print(est.summary())

# FE
X_fe = df_pic_X.copy(deep=True)
X_fe = X_fe.set_index(ID_cols)
X_fe = sm.add_constant(X_fe)

y_fe = df_pic[ID_cols+["WAR"]].copy(deep=True)
y_fe = y_fe.set_index(ID_cols)

mod = PanelOLS(y_fe, X_fe, 
               entity_effects=True, 
               drop_absorbed=True,
               check_rank =False
              )
res = mod.fit(cov_type='clustered', cluster_entity=True)
print(res)

                            OLS Regression Results                            
Dep. Variable:                    WAR   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                     39.07
Date:                Sun, 30 May 2021   Prob (F-statistic):          1.49e-243
Time:                        14:18:15   Log-Likelihood:                -5359.2
No. Observations:                3007   AIC:                         1.080e+04
Df Residuals:                    2965   BIC:                         1.105e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

C:\Users\Daguan\miniconda3\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

POS_1B, POS_2B, POS_3B, POS_C, POS_DH, POS_OF

  warnings.warn(


                          PanelOLS Estimation Summary                           
Dep. Variable:                    WAR   R-squared:                        0.3720
Estimator:                   PanelOLS   R-squared (Between):             -0.2350
No. Observations:                3007   R-squared (Within):               0.3720
Date:                Sun, May 30 2021   R-squared (Overall):              0.1868
Time:                        14:18:16   Log-likelihood                   -4556.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      40.273
Entities:                         592   P-value                           0.0000
Avg Obs:                       5.0794   Distribution:                 F(35,2380)
Min Obs:                       1.0000                                           
Max Obs:                       14.000   F-statistic (robust):             31.626
                            